?  =  variable name, possibly with indexing/slicing  
??? = function / class name  
?????? = complex expression containing variables, functions etc  
???B??? = complex expression that must contain "B"  

# What are most of the neurons "talking about"?

# Section 1. Load everything again. 

In [ ]:
import os
import numpy as np
from scipy import io
from scipy import stats
from scipy.ndimage import gaussian_filter
from ??? import ??? as ??? # import the plotting function
%matplotlib inline

In [ ]:
# rerun all the prep and processing from mesoscope1.ipynb
mname, datexp, blk = 'TX39', '2019_05_31', '1' 
root      = '/home/neuraldata/data/meso'
mov    = np.load(os.path.join(root, 'mov.npy')) 
iframe    = np.load(os.path.join(root, 'iframe.npy')) 
ops = np.load(os.path.join(root, 'suite2p', 'combined', 'ops.npy'), allow_pickle=True).item()
spks = np.load(os.path.join(root, 'suite2p', 'combined', 'spks.npy'))
stat = np.load(os.path.join(root, 'suite2p', 'combined', 'stat.npy'), allow_pickle=True)
ypos = [stat[k]['med'][0] for k in range(len(stat))]
xpos = [stat[k]['med'][1] for k in range(len(stat))]
ypos, xpos = np.array(ypos), np.array(xpos)
ypos, xpos = ypos/.5, xpos/.75 
dt = 1 # time offset 
ivalid = iframe+dt<spks.shape[-1] # remove timepoints outside the valid time range
iframe = iframe[ivalid]
mov = mov[:, :, ivalid]
S = spks[:, iframe+dt]
S = stats.zscore(S, axis=1) # z-score the neural activity before doing anything
print('total neurons %d'%len(stat))
print('recorded from an area of %2.2f um by %2.2f um'%(np.ptp(ypos), np.ptp(xpos)))

# Section 2. Run PCA

In [ ]:
from sklearn.decomposition import PCA
pca_model = PCA(??? = ?) # PCA is a class. Here we define a model with 100 components
pca_model.???(S) # here we apply the model to the data
# most models in sklearn work this way (linear regression, LASSO, ICA, tSNE)

In [ ]:
# plot some components
X = pca_model.??? # find the variable that holds the principal components

plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
for j in range(5):
    plt.plot(X[j] - j * .05)
    
plt.subplot(1,2,2)
time_range = np.???(?, ?) # make an array containing the time range 2000 to 2500
for j in range(5):
    plt.plot(X[j, time_range] - j * .05)
plt.show()

In [ ]:
# compute the autocorrelogram of these components 
from numpy.fft import fft, ifft

# this function uses the Fourier trick to compute the autocorrelogram
def autocorr(X, axis=-1):    
    fX  = fft(X, axis=axis)
    acc = np.real(ifft(fX * np.conj(fX), axis=axis))
    return acc

X   = pca_model.components_
acc = ???(?) # apply the autocorr function to the principal components

# sample some colors from the magma colormap
num_PC = 20 # plot this many PCs
colormap = plt.cm.magma
colors   = colormap(np.linspace(0, 1,num_PC))

# bright colors are later components
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
for j in range(num_PC):
    plt.plot(?, c = colors[j]) # plot the first 50 timepoints of the autocorrelogram for component j
plt.ylabel('correlation')
plt.xlabel('time lag')

plt.subplot(1,2,2)
for j in range(num_PC):
    plt.???(np.arange(1,10000), acc[j, 1:10000], c = colors[j]) # make a plot on a logarithmic X scale
plt.xlabel('time lag (log scale)')
plt.show()

In [ ]:
# color neurons according to their PC coefficients
plt.figure(figsize=(16,9))
for j in range(6):
    pc0 = ???S???pca_model??? # multiply each neuron with PC #j to get its coefficient
    pc0 /= np.max(np.abs(pc0)) # normalize components to max(abs) = 1
    lam = np.abs(pc0) # make the size of each dot proportional to abs(coeff)
    plt.subplot(2,3,1+j)
    plt.scatter(xpos, -ypos, s= 10 * lam, c = pc0, cmap='bwr', ??? = ?, vmin=-1, vmax=1) # set the transparency to 0.5
    if j==0:
        plt.colorbar()
    plt.title('PC %d'%(1+j))
    plt.axis('off')

In [ ]:
# compute receptive fields for the PCs. 
Sp = model.components_
Sp = stats.zscore(Sp, axis=1)

# below is the same code as in mesoscope1.ipynb. Only one variable needs to change in one place
ly, lx, NT = mov.shape
X  = np.reshape(mov - 0.5, [-1, NT]) # reshape to Npixels by Ntimepoints
X  = stats.zscore(np.abs(X), axis=1)/NT**.5  # z-score each pixel separately
B0 = X @ S.T # get the receptive fields for each neuron
B0 = np.reshape(B0, [mov.shape[0], mov.shape[1], -1])
B0 = gaussian_filter(B0, [.5, .5, 0]) # smooth each receptive field a little

In [ ]:
plt.figure(figsize=(18, 8))
rfmax = np.max(np.abs(B0))
for j in range(40):
    plt.subplot(5,8,j+1)
    rf = ? # the receptive field of component j
    plt.imshow(rf, aspect='auto', cmap = 'bwr', vmin = -rfmax, vmax = rfmax) # plot the receptive field for each neuron
    plt.title('PC %d'%(1+j))
    plt.axis('off')
    
plt.show()    

## Two new mysteries: 
1) why do the biggest components have no receptive fields  
2) why are the receptive fields so distributed

In [ ]:
# this cell is just a demonstration, no exercise

# solve the second mystery: PCA of translation-invariant data always looks like that
# make some random noise and smooth it
nPC = 40

Xn = np.random.randn(10000, 32,32)
Xn = gaussian_filter(Xn, [0, 2, 2])
Xn = np.reshape(Xn, (10000, -1))
model = PCA(n_components= nPC).fit(Xn)

bPC = np.reshape(model.components_.T, (32,32, nPC))
plt.figure(figsize=(14, 8))
rfmax = np.max(bPC)
for j in range(nPC):
    plt.subplot(5,8,j+1)
    rf = bPC[:,:,j]
    plt.imshow(rf, aspect='auto', cmap = 'bwr', vmin = -rfmax, vmax = rfmax) # plot the receptive field for each neuron
    plt.title('PC %d'%(1+j))    
    plt.axis('off')
plt.show()    

# compare with DCT https://en.wikipedia.org/wiki/Discrete_cosine_transform
del Xn    

# Section 3. Run Rastermap 

Rastermap re-arranges neurons in the raster plot based on similarity of activity

In [ ]:
from rastermap import Rastermap
# use Rastermap exactly how you used PCA above. 
# this time we want only ONE component, but we want 100 nodes (n_X=100)
model = ???n_X=100???
model.??????

### Sort and smooth neurons along the one-dimensional embedding

In [ ]:
# this function performs a running average filter over the first dimension of X
def running_average(X, nbin = 50):
    Y = np.cumsum(X, axis=0)
    Y = Y[nbin:, :] - Y[:-nbin, :] # trick to do efficient box filter
    return Y

isort = np.argsort(model.embedding[:,0])
Sfilt = running_average(S[isort, :], 50)
Sfilt = stats.zscore(Sfilt, axis=1)

In [ ]:
# display the smoothed rastermap, show only timepoints "time_range" (take a slice)

time_range = np.arange(2000,2500)
#time_range = np.arange(12000,12500)

plt.figure(figsize=(16,9))
plt.imshow(?, vmin = -1, vmax=2, aspect='auto', cmap='gray_r')
plt.xlabel('time points')
plt.ylabel('sorted neurons')
plt.show()

In [ ]:
# HERE PLOT cells according to their rastermap embedding
lam = model.lam # this is how well each cell correlates with assigned component
lam = lam / np.max(lam) # normalize lam to max of 1

plt.figure(figsize=(8,6))
# fancy plot: scatter xpos and -ypos, color each dot by the model emmbedding position, using the colormap gist_ncar, 
# with a transparency value of 0.5, using a dot size proportional to lam
??????
plt.colorbar()

## Do the Rastermap components have receptive fields?

In [ ]:
# Sfilt contains neural activity smoothed across neurons.
# Pick every 50th row . It will contain the average of 100 neurons with similar activity. 
Sp = ??????

ly, lx, nstim = mov.shape

NN, NT = S.shape 

X = np.reshape(mov, [-1, NT]) # reshape to Npixels by Ntimepoints
X = stats.zscore(np.abs(X-0.5), axis=1)/NT**.5  # z-score each pixel separately
npix = X.shape[0]

lam = .01
ncomps = Sp.shape[0]
B0 = X @ Sp.T # get the receptive fields for each neuron
B0 = np.reshape(B0, [mov.shape[0], mov.shape[1], ncomps])

In [ ]:
# now plot these receptive fields like we did before
??????
??????
??????
??????
??????
??????